# This notebook aims to get human TCPS dTE sequences

In [1]:
from pyfaidx import Fasta
import pandas as pd
from Bio.Seq import Seq
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO, pairwise2
from itertools import combinations
from multiprocessing import Pool, cpu_count
from pathlib import Path

/home/lucy/anaconda3/envs/qc_pipe/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
#from pairwise2 import format_alignment

In [4]:
import time

In [5]:
import sequence_comparison as seqComp

In [6]:
from gtfparse import read_gtf

In [7]:
import os
import gc

## load data

the DESeq individual loci table

In [9]:
indiv_dTE = pd.read_csv("/home/lucy/Dropbox/TCPS_TE_results/TE_indi_deseq.tsv", delimiter='\t', header = 0 )
indiv_dTE.head()

,TE name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,L2,0.097405,0.197787,1.457064,0.135744,0.892024,0.999991
1,AluSg,0.493240,1.038439,0.909837,1.141346,0.253726,0.999991
2,L1PA8A,1.504698,-0.263127,0.834217,-0.315418,0.752444,0.999991
3,L1MDb,0.179092,0.166453,1.381934,0.120449,0.904127,0.999991
4,AluSp,0.033562,0.514654,3.312685,0.155358,0.876539,0.999991


In [10]:
indiv_dTE.iloc[500:520]['TE name']

500     AluSz_199
501      MIRb_340
502       L2b_218
503     AluSz_200
504     AluJb_315
505      AluSq4_7
506    AluSx1_297
507      AluSq_63
508     AluJr_171
509    AluSx1_298
510      AluSg_82
511       MIR_282
512     AluSz6_90
513     AluJo_210
514         L2_97
515      HAL1ME_2
516      HAL1ME_3
517         L2_98
518     AluJb_324
519     AluSz_204
Name: TE name, dtype: object

remove coding genes in the dTE df

In [10]:
indiv_dTE = indiv_dTE[ ~indiv_dTE['TE name'].str.startswith('ENSG')]

In [11]:
indiv_dTE.shape

(172873, 7)

#### load Sandy's GTF and change to df format

In [15]:
# Read the GTF file into a Pandas DataFrame
df = read_gtf("/home/lucy/Dropbox/TCPS_TE_results/hg38_gencode_rmsk_indi.gtf")
df.head()

# Filter the DataFrame, e.g., to get only gene entries
#genes_df = df[df["feature"] == "gene"]
#print(genes_df.head())

INFO:root:Extracted GTF attributes: ['gene_id']


seqname,source,feature,start,end,score,strand,frame,gene_id
cat,cat,cat,i64,i64,f32,cat,u32,str
"""chr1""","""hg38_rmsk""","""exon""",67108754,67109046,1892.0,"""+""",0,"""L1P5"""
"""chr1""","""hg38_rmsk""","""exon""",8388316,8388618,2582.0,"""-""",0,"""AluY"""
"""chr1""","""hg38_rmsk""","""exon""",25165804,25166380,4085.0,"""+""",0,"""L1MB5"""
"""chr1""","""hg38_rmsk""","""exon""",33554186,33554483,2285.0,"""-""",0,"""AluSc"""
"""chr1""","""hg38_rmsk""","""exon""",41942895,41943205,2451.0,"""-""",0,"""AluY_1"""


In [30]:
print( df.columns ) #= 
df.columns = ['genoName', 'source', 'feature', 'genoStart', 'genoEnd', 'score', 'strand', 'frame', 'gene_id']

['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'gene_id']


In [32]:
df.head()

genoName,source,feature,genoStart,genoEnd,score,strand,frame,gene_id
cat,cat,cat,i64,i64,f32,cat,u32,str
"""chr1""","""hg38_rmsk""","""exon""",67108754,67109046,1892.0,"""+""",0,"""L1P5"""
"""chr1""","""hg38_rmsk""","""exon""",8388316,8388618,2582.0,"""-""",0,"""AluY"""
"""chr1""","""hg38_rmsk""","""exon""",25165804,25166380,4085.0,"""+""",0,"""L1MB5"""
"""chr1""","""hg38_rmsk""","""exon""",33554186,33554483,2285.0,"""-""",0,"""AluSc"""
"""chr1""","""hg38_rmsk""","""exon""",41942895,41943205,2451.0,"""-""",0,"""AluY_1"""


make a dictionary to map the genome location information with gene ids

In [36]:
rmsk_df = pd.DataFrame( df, columns=df.columns)

In [37]:
rmsk_df.head()

,genoName,source,feature,genoStart,genoEnd,score,strand,frame,gene_id
0,chr1,hg38_rmsk,exon,67108754,67109046,1892.0,+,0,L1P5
1,chr1,hg38_rmsk,exon,8388316,8388618,2582.0,-,0,AluY
2,chr1,hg38_rmsk,exon,25165804,25166380,4085.0,+,0,L1MB5
3,chr1,hg38_rmsk,exon,33554186,33554483,2285.0,-,0,AluSc
4,chr1,hg38_rmsk,exon,41942895,41943205,2451.0,-,0,AluY_1


#### load the genome annotation df directly

In [16]:
rmsk_df = pd.read_csv("/home/lucy/Dropbox/TCPS_TE_results/hg38_gencode_rmsk_indi.csv", header = 0, index_col= 0 )

## Get top differential TEs and extract fasta
- p_adj threshold: 0.05, log2FC threshold: +/-2

### extract significant dTEs

In [12]:
indiv_dTE.head()

,TE name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,L2,0.097405,0.197787,1.457064,0.135744,0.892024,0.999991
1,AluSg,0.493240,1.038439,0.909837,1.141346,0.253726,0.999991
2,L1PA8A,1.504698,-0.263127,0.834217,-0.315418,0.752444,0.999991
3,L1MDb,0.179092,0.166453,1.381934,0.120449,0.904127,0.999991
4,AluSp,0.033562,0.514654,3.312685,0.155358,0.876539,0.999991


In [13]:
indiv_dTE_sig = indiv_dTE[indiv_dTE['padj'] < 0.05 ]

### extract genome location information for the dTEs

In [17]:
indiv_dTE_sig.shape

(189, 7)

In [18]:
rmsk_df_selected = rmsk_df[ np.isin( rmsk_df['gene_id'] ,indiv_dTE_sig['TE name']) ]

In [19]:
rmsk_df_selected.shape

(189, 9)

In [20]:
import gc

In [21]:
gc.collect()

1402

In [22]:
genoLoc_ls = [ list( rmsk_df_selected.loc[i, ['genoName', 'genoStart', 'genoEnd', 'strand']])  for i in rmsk_df_selected.index ]

In [23]:
genoLoc_ls[0]

['chr1', 21875471, 21875616, '-']

In [24]:
genoLoc_dict = dict( zip( rmsk_df_selected['gene_id'] , genoLoc_ls ))

In [25]:
geno_loc_df = pd.DataFrame( index = indiv_dTE_sig.index, columns=['genoName', 'genoStart', 'genoEnd', 'strand'] )

In [26]:
rmsk_df_selected.shape

(189, 9)

In [81]:
indiv_dTE_sig.shape

(189, 7)

In [27]:
for i in indiv_dTE_sig.index:
    k = indiv_dTE_sig.loc[i, 'TE name' ]
    geno_loc_df.loc[i] = genoLoc_dict[k]
    

In [28]:
indiv_dTE_sig = indiv_dTE_sig.merge( geno_loc_df, left_index=True, right_index=True)

In [29]:
indiv_dTE_sig.head()

,TE name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genoName,genoStart,genoEnd,strand
2361,MIRb_2591,21.977249,2.001854,0.495015,4.044027,0.000053,0.008073,chr1,21875471,21875616,-
2983,MIRb_3645,10.310570,2.238395,0.535974,4.176310,0.000030,0.005203,chr1,26188068,26188211,+
2992,LTR10A_6,7.215176,4.566861,1.173587,3.891371,0.000100,0.013165,chr1,26268832,26269405,+
3068,MIR_2537,1.101041,3.860287,1.037415,3.721063,0.000198,0.022386,chr1,26554369,26554498,+
3286,AluSp_907,2.735933,3.134858,0.762647,4.110499,0.000039,0.006472,chr1,27730174,27730485,-


### extract up and down regulated TEs

In [64]:
print( indiv_dTE.shape)
print( indiv_dTE_sig.shape)

(172873, 7)
(189, 7)


In [88]:
ser_up = indiv_dTE_sig[ indiv_dTE_sig['log2FoldChange'] > 2 ] 
ad_up = indiv_dTE_sig[ indiv_dTE_sig['log2FoldChange'] < -2  ] 

print( ser_up.shape)
print( ad_up.shape)

(133, 11)
(9, 11)


In [91]:
ser_up.sort_values( by = 'log2FoldChange', ascending= False, inplace= True )
ad_up.sort_values( by = 'log2FoldChange', ascending= True, inplace= True )

/tmp/ipykernel_184823/774895865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser_up.sort_values( by = 'log2FoldChange', ascending= False, inplace= True )
/tmp/ipykernel_184823/774895865.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ad_up.sort_values( by = 'log2FoldChange', ascending= True, inplace= True )


In [92]:
ser_up.head()

,TE name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genoName,genoStart,genoEnd,strand
83529,L2a_91495,1.966883,5.273339,1.484466,3.552346,3.818120e-04,0.036724,chr9,68735690,68736273,-
64392,AluJb_45906,1.583823,5.024861,1.279641,3.926772,8.610000e-05,0.011711,chr6,152722736,152723016,-
70722,L1ME3G_7149,3.446495,4.905536,0.814057,6.026035,1.680000e-09,0.000002,chr7,99081123,99081223,+
61586,LTR10A_115,6.645893,4.811246,1.268065,3.794162,1.481430e-04,0.017776,chr6,87213284,87213856,-
6582,L2b_3755,2.860923,4.731285,1.057905,4.472316,7.740000e-06,0.001795,chr1,54992141,54992205,+


In [93]:
ad_up.head()

,TE name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genoName,genoStart,genoEnd,strand
108880,L2c_97668,10.632930,-3.965963,1.073278,-3.695186,0.000220,0.024233,chr11,102520550,102520704,-
139569,L2b_80878,8.499507,-3.556407,0.986942,-3.603462,0.000314,0.031881,chr16,50612862,50613288,-
142728,AluJo_63739,6.856328,-3.441202,0.975172,-3.528816,0.000417,0.039317,chr16,89762643,89762805,-
150077,MSTB1_4089,23.010690,-2.940496,0.780072,-3.769518,0.000164,0.019275,chr17,65541178,65541243,+
10902,AluSz_5511,35.757835,-2.470471,0.587304,-4.206457,0.000026,0.004672,chr1,151367072,151367403,+


### save the top dTE fasta
- ad-up ( all is 9 )
- ser-up top 10, 20, 50, last 50

In [96]:
ad_up.columns

Index(['TE name', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue',
       'padj', 'genoName', 'genoStart', 'genoEnd', 'strand'],
      dtype='object')

In [97]:
region_df_cols = ['genoName', 'genoStart', 'genoEnd', 'strand', 'TE name' ]

In [94]:
hg38_ref_path = '/home/lucy/Dropbox/human_ref_genome/hg38_grch38/hg38.fa'

In [99]:
seqComp.extract_ucsc_bedlike_regions( genome_fasta= hg38_ref_path,
                                     regions_df= ad_up[region_df_cols ], 
                                     output_fasta='./sequence_similarity_input/' + 'tcps_ad_up_all.fasta') 

✅ Extracted sequences saved to: ./sequence_similarity_input/tcps_ad_up_all.fasta


In [119]:
element_ranks = [ [0,10], [0,20], [0,50] , [-50, ser_up.shape[0]] ]
save_file_names = [ f'tcps_ser_up_top{x}.fasta' for x in [10,20,50] ] + ['tcps_ser_up_last50.fasta']

In [114]:
element_ranks[-1]

[-50, 133]

In [120]:
save_file_names

['tcps_ser_up_top10.fasta',
 'tcps_ser_up_top20.fasta',
 'tcps_ser_up_top50.fasta',
 'tcps_ser_up_last50.fasta']

In [118]:
#ser_up.tail()

In [117]:
#ser_up.iloc[ element_ranks[-1][0]:element_ranks[-1][1] ].tail()

In [121]:
for i,e in enumerate( element_ranks):
    seqComp.extract_ucsc_bedlike_regions( genome_fasta= hg38_ref_path,
                                     regions_df= ser_up[region_df_cols ].iloc[e[0]: e[1]], 
                                     output_fasta='./sequence_similarity_input/' + save_file_names[i]) 

✅ Extracted sequences saved to: ./sequence_similarity_input/tcps_ser_up_top10.fasta
✅ Extracted sequences saved to: ./sequence_similarity_input/tcps_ser_up_top20.fasta
✅ Extracted sequences saved to: ./sequence_similarity_input/tcps_ser_up_top50.fasta
✅ Extracted sequences saved to: ./sequence_similarity_input/tcps_ser_up_last50.fasta


## calculate similarity scores for the extracted fasta files

In [123]:
gc.collect()

25379

In [125]:
fnames = pd.Series( os.listdir("./sequence_similarity_input/") )
fnames = fnames[ fnames.str.startswith('tcps') ]
fnames

6      tcps_ser_up_top10.fasta
14        tcps_ad_up_all.fasta
15    tcps_ser_up_last50.fasta
20     tcps_ser_up_top20.fasta
23     tcps_ser_up_top50.fasta
dtype: object

In [126]:
input_dir = './sequence_similarity_input/'

In [128]:
for f in fnames:
    print( f'Processing {f}')
    TIME_S = time.time()
    seqComp.process_group(input_dir + f, out_dir="sequence_similarity_results", n_jobs=4,plot_heatmap=True )
    TIME_E = time.time()
    TOTAL_TIME = (TIME_E - TIME_S)/60
    print( f"Time used: {TOTAL_TIME:.1f} min" )

Processing tcps_ser_up_top10.fasta
Loaded 10 sequences from ./sequence_similarity_input/tcps_ser_up_top10.fasta
Average identity: 37.02% ± 17.54%
Time used: 0.0 min
Processing tcps_ad_up_all.fasta
Loaded 9 sequences from ./sequence_similarity_input/tcps_ad_up_all.fasta
Average identity: 37.40% ± 14.68%
Time used: 0.0 min
Processing tcps_ser_up_last50.fasta
Loaded 50 sequences from ./sequence_similarity_input/tcps_ser_up_last50.fasta
Average identity: 34.32% ± 18.56%
Time used: 0.1 min
Processing tcps_ser_up_top20.fasta
Loaded 20 sequences from ./sequence_similarity_input/tcps_ser_up_top20.fasta
Average identity: 45.15% ± 19.30%
Time used: 0.0 min
Processing tcps_ser_up_top50.fasta
Loaded 50 sequences from ./sequence_similarity_input/tcps_ser_up_top50.fasta
Average identity: 41.47% ± 16.96%
Time used: 0.1 min


## view result

In [8]:
tcps_ser_up_top10_result = pd.read_csv("./sequence_similarity_results/tcps_ser_up_top10_pairwise_identity.csv", header = 0 , index_col = None)
tcps_ser_up_top10_result.head()

,ID1,ID2,PercentIdentity,Alignment_str
0,L2a_91495,AluJb_45906,40.480274,ctttttaactggtctttctacttctgcaacattca-gagtga--cc...
1,L2a_91495,L1ME3G_7149,17.152659,ctttttaactggtctttctacttctgcaacattcagagtgaccaga...
2,L2a_91495,LTR10A_115,64.837050,cttt-ttaac-tg---gt---c------tttctacttctgcaacat...
3,L2a_91495,L2b_3755,10.977702,ctttttaactggtctttctacttctgcaacattcagagtgaccaga...
4,L2a_91495,L2a_106449,13.722127,ctttttaactggtctttctacttctgcaacattcagagtgaccaga...


In [41]:
indiv_dTE_sig.head()

,TE name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,genoName,genoStart,genoEnd,strand
2361,MIRb_2591,21.977249,2.001854,0.495015,4.044027,0.000053,0.008073,chr1,21875471,21875616,-
2983,MIRb_3645,10.310570,2.238395,0.535974,4.176310,0.000030,0.005203,chr1,26188068,26188211,+
2992,LTR10A_6,7.215176,4.566861,1.173587,3.891371,0.000100,0.013165,chr1,26268832,26269405,+
3068,MIR_2537,1.101041,3.860287,1.037415,3.721063,0.000198,0.022386,chr1,26554369,26554498,+
3286,AluSp_907,2.735933,3.134858,0.762647,4.110499,0.000039,0.006472,chr1,27730174,27730485,-


In [42]:
indiv_dTE_sig2 = indiv_dTE_sig.set_index( 'TE name' ,inplace= False )

In [49]:
indiv_dTE_sig2.loc['LTR10A_115']

baseMean          6.645893
log2FoldChange    4.811246
lfcSE             1.268065
stat              3.794162
pvalue            0.000148
padj              0.017776
genoName              chr6
genoStart         87213284
genoEnd           87213856
strand                   -
Name: LTR10A_115, dtype: object

In [50]:
indiv_dTE_sig2.loc['LTR10A_6']

baseMean          7.215176
log2FoldChange    4.566861
lfcSE             1.173587
stat              3.891371
pvalue              0.0001
padj              0.013165
genoName              chr1
genoStart         26268832
genoEnd           26269405
strand                   +
Name: LTR10A_6, dtype: object

In [43]:
adjusted_top10_result = seqComp.add_seq_comparison_info( tcps_ser_up_top10_result, indiv_dTE_sig2, merged_result= True )

In [44]:
adjusted_top10_result.sort_values( by = 'adj_percentIdentity', ascending= False , inplace=True)

In [45]:
adjusted_top10_result.head()

,ID1,ID2,PercentIdentity,Alignment_str,seq1_len,seq2_len,total_match,adj_percentIdentity
26,LTR10A_115,LTR10A_6,88.481675,tgttaaatatagtgaaccccaagtttctcttcaaagaatcagtatg...,572,573,507,88.558952
14,AluJb_45906,AluJo_14923,72.569444,t----a--a---t---tc-ca---g----c--a-c--tttgg-gag...,280,288,209,73.591549
2,L2a_91495,LTR10A_115,64.837050,cttt-ttaac-tg---gt---c------tttctacttctgcaacat...,583,572,378,65.454545
5,L2a_91495,LTR10A_6,64.665523,cttt-ttaactg-gt---c------tttctacttctgcaacat---...,583,573,377,65.224913
30,L2b_3755,L2a_106449,57.500000,--c---gtgcc-t-tctccag-agt-ct-tatgca-gt-g--cctg...,64,80,46,63.888889


In [51]:
%%capture output
# I love this magic command 
for id1, id2, score, aln in adjusted_top10_result.iloc[0:10, 0:4].values:
    print( f"Comparing sequences of score {score:.2f}:\n\t{id1}\n\t{id2}")
    aln_ls = aln.split("\n")
    seqComp.alignment_str_parser( aln_ls, line_width = 100, )

In [52]:
with open('./sequence_similarity_results/tcps_ser_up_top10_seq_aln.txt', 'w') as f:
        f.write(output.stdout)

## save data

In [129]:
rmsk_df.to_csv("/home/lucy/Dropbox/TCPS_TE_results/hg38_gencode_rmsk_indi.csv", header = True, index= True )

In [32]:
indiv_dTE_sig.to_csv('genoLoc_TCPS_Deseq_indiv_locus_result.csv', header = True, index = True )

In [33]:
logi = pd.read_csv('OUTPUT_EXPLAIN.csv', header = 0, index_col = 0 )

In [34]:
logi.columns

Index(['count_table_indiv.csv', 'count_table_subF.csv', 'mm39_rmsk_full.tsv',
       'Deseq_indiv_locus_results.csv', 'genoLoc_Deseq_indiv_locus_result.csv',
       'compare_dTE_overlap/'],
      dtype='object')

In [36]:
logi['genoLoc_Deseq_indiv_locus_result.csv'].loc['explain']

'Derived from the Deseq_indiv_locus_results csv file. Added genome location information mapped from rmsk. Note that there are more rows in this table than in the original deseq table because some transcript ID can map to multiple genome locations'

In [37]:
logi['genoLoc_TCPS_Deseq_indiv_locus_result.csv'] = ['5a' , 'NA', 'Derived from the TCPS Deseq individual locus result stored in Dropbox --> TCPS_TE_results --> TE_indi_deseq.tsv. Genome location information added based on the GTF file Sandy shared which can be found in Dropbox TCPS_TE_results/hg38_gencode_rmsk_indi'] 

In [38]:
logi.columns

Index(['count_table_indiv.csv', 'count_table_subF.csv', 'mm39_rmsk_full.tsv',
       'Deseq_indiv_locus_results.csv', 'genoLoc_Deseq_indiv_locus_result.csv',
       'compare_dTE_overlap/', 'genoLoc_TCPS_Deseq_indiv_locus_result.csv'],
      dtype='object')

In [39]:
logi.to_csv("./OUTPUT_EXPLAIN.csv", header = True, index = True )